In [1]:
import torch
from torchvision import (
    datasets,
    transforms
)
from torch.utils import data

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
plt.rcParams["font.family"] = "monospace"

In [2]:
if torch.cuda.is_available():
    print("CUDA found, bounding...")
    DEVICE = torch.device('cuda:0')
else:
    print("NO CUDA found, bounding to CPU...")
    DEVICE = torch.device('cpu')

CUDA found, bounding...


In [3]:
torch.cuda.get_device_properties(DEVICE)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1650', major=7, minor=5, total_memory=4095MB, multi_processor_count=16)

In [4]:
DTYPE = torch.float32

# Elementwise Operations
A: (M, N)
B: (M, N)
Result: (M, N)

-> Shapes MUST match
Every elements is --operation-- in the same location
ADDING, SUBTRACTING, MULTIPLYING, DIVIDING etc...

In [5]:
A = torch.rand((5, 5), dtype=DTYPE, device=DEVICE)
A, A.shape

(tensor([[0.0728, 0.4616, 0.5291, 0.8198, 0.3602],
         [0.5139, 0.2587, 0.5460, 0.2750, 0.1266],
         [0.6231, 0.6601, 0.9001, 0.2673, 0.1146],
         [0.8004, 0.7444, 0.1266, 0.1549, 0.3384],
         [0.9149, 0.9257, 0.4855, 0.5305, 0.7519]], device='cuda:0'),
 torch.Size([5, 5]))

In [6]:
B = torch.rand((5, 5), dtype=DTYPE, device=DEVICE)
B, B.shape

(tensor([[0.5916, 0.8713, 0.7500, 0.6825, 0.6431],
         [0.8006, 0.4907, 0.2482, 0.0362, 0.0552],
         [0.4742, 0.4735, 0.8756, 0.8402, 0.5302],
         [0.3152, 0.0488, 0.8205, 0.7962, 0.6095],
         [0.9069, 0.8260, 0.3349, 0.0318, 0.3484]], device='cuda:0'),
 torch.Size([5, 5]))

In [9]:
# Adding
AB = A + B
AB, AB.shape

(tensor([[0.6644, 1.3330, 1.2791, 1.5023, 1.0033],
         [1.3145, 0.7494, 0.7942, 0.3112, 0.1818],
         [1.0973, 1.1335, 1.7756, 1.1075, 0.6448],
         [1.1157, 0.7932, 0.9471, 0.9512, 0.9479],
         [1.8218, 1.7517, 0.8204, 0.5623, 1.1004]], device='cuda:0'),
 torch.Size([5, 5]))

In [10]:
(A + B).all() == torch.add(A, B).all()

tensor(True, device='cuda:0')

In [12]:
# Subtracting
AB = torch.sub(A, B)
AB, AB.shape

(tensor([[-0.5188, -0.4097, -0.2209,  0.1373, -0.2829],
         [-0.2867, -0.2319,  0.2979,  0.2388,  0.0714],
         [ 0.1489,  0.1866,  0.0245, -0.5729, -0.4156],
         [ 0.4852,  0.6957, -0.6939, -0.6413, -0.2711],
         [ 0.0081,  0.0998,  0.1505,  0.4987,  0.4035]], device='cuda:0'),
 torch.Size([5, 5]))

In [13]:
torch.sub(A, B).all() == (A - B).all()

tensor(True, device='cuda:0')

In [15]:
# Multiplying
AB = torch.mul(A, B)
AB, AB.shape

(tensor([[0.0431, 0.4023, 0.3969, 0.5596, 0.2317],
         [0.4114, 0.1270, 0.1355, 0.0100, 0.0070],
         [0.2954, 0.3125, 0.7881, 0.2246, 0.0608],
         [0.2523, 0.0363, 0.1039, 0.1234, 0.2063],
         [0.8297, 0.7646, 0.1626, 0.0169, 0.2620]], device='cuda:0'),
 torch.Size([5, 5]))

In [19]:
torch.mul(A, B).all() == (A * B).all()

tensor(True, device='cuda:0')

In [21]:
# Dividing
AB = torch.div(A, B)
AB, AB.shape

(tensor([[ 0.1231,  0.5298,  0.7055,  1.2011,  0.5601],
         [ 0.6419,  0.5273,  2.2002,  7.5961,  2.2938],
         [ 1.3140,  1.3941,  1.0279,  0.3182,  0.2162],
         [ 2.5391, 15.2641,  0.1543,  0.1946,  0.5552],
         [ 1.0089,  1.1208,  1.4495, 16.6668,  2.1580]], device='cuda:0'),
 torch.Size([5, 5]))

In [22]:
torch.div(A, B).all() == (A / B).all()

tensor(True, device='cuda:0')

In [24]:
# Exponential
torch.e ** A

tensor([[1.0755, 1.5867, 1.6975, 2.2701, 1.4336],
        [1.6718, 1.2953, 1.7264, 1.3165, 1.1349],
        [1.8647, 1.9349, 2.4597, 1.3064, 1.1214],
        [2.2265, 2.1053, 1.1350, 1.1676, 1.4027],
        [2.4966, 2.5237, 1.6249, 1.6997, 2.1211]], device='cuda:0')

In [26]:
torch.exp(A).all() == (torch.e ** A).all()

tensor(True, device='cuda:0')

In [28]:
# Square Root
A ** .5

tensor([[0.2698, 0.6794, 0.7274, 0.9054, 0.6002],
        [0.7169, 0.5087, 0.7389, 0.5244, 0.3558],
        [0.7894, 0.8124, 0.9487, 0.5170, 0.3385],
        [0.8947, 0.8628, 0.3558, 0.3936, 0.5817],
        [0.9565, 0.9621, 0.6968, 0.7283, 0.8671]], device='cuda:0')

In [29]:
torch.sqrt(A).all() == (A ** .5).all()

tensor(True, device='cuda:0')

What happens if shapes DO NOT MATCH?

v: (N) -> N elements
M: (M, N) -> M rows, N columns

Broadcasting -> "Strecthing" a lower dimension so it matches with the other.

V: (M, N)

EXAMPLE

v: [[0., 1., 2., 3., 4.]] -> (1, 5)

M: [0., 1., 2., 0., 5.], -> (3, 5)
   [3., 0., 1., 1., 1.],
   [10., 3., 2., 1., 4.]

After Broadcasting:

V: [0., 1., 2., 3., 4.], -> (3, 5)
   [0., 1., 2., 3., 4.],
   [0., 1., 2., 3., 4.]

Result:

[0., 1., 4., 0., 20.],
[0., 0., 2., 3., 4.],
[0., 3., 4., 3., 16.]

In [30]:
v = torch.arange(0, 5, 1, dtype=DTYPE, device=DEVICE)
v, v.shape, v.ndim

(tensor([0., 1., 2., 3., 4.], device='cuda:0'), torch.Size([5]), 1)

In [31]:
M = torch.tensor([
    [0., 1., 2., 0., 5.],
    [3., 0., 1., 1., 1.],
    [10., 3., 2., 1., 4.]
], dtype=DTYPE, device=DEVICE)
M, M.shape, M.ndim

(tensor([[ 0.,  1.,  2.,  0.,  5.],
         [ 3.,  0.,  1.,  1.,  1.],
         [10.,  3.,  2.,  1.,  4.]], device='cuda:0'),
 torch.Size([3, 5]),
 2)

In [32]:
Mv = torch.mul(M, v)
Mv, Mv.shape, Mv.ndim

(tensor([[ 0.,  1.,  4.,  0., 20.],
         [ 0.,  0.,  2.,  3.,  4.],
         [ 0.,  3.,  4.,  3., 16.]], device='cuda:0'),
 torch.Size([3, 5]),
 2)

# Transpose of a Tensor
A: (M, N) -> M rows, N columns

Transposing: Swaping dimensions with each other.
(Rows become Columns, Columns become Rows)

A.T: (N, M)

In [33]:
M, M.shape

(tensor([[ 0.,  1.,  2.,  0.,  5.],
         [ 3.,  0.,  1.,  1.,  1.],
         [10.,  3.,  2.,  1.,  4.]], device='cuda:0'),
 torch.Size([3, 5]))

In [34]:
MT = M.T
MT, MT.shape

(tensor([[ 0.,  3., 10.],
         [ 1.,  0.,  3.],
         [ 2.,  1.,  2.],
         [ 0.,  1.,  1.],
         [ 5.,  1.,  4.]], device='cuda:0'),
 torch.Size([5, 3]))

In [35]:
v, v.shape

(tensor([0., 1., 2., 3., 4.], device='cuda:0'), torch.Size([5]))

In [42]:
v = v.reshape(1, 5)
v, v.shape, v.ndim

(tensor([[0., 1., 2., 3., 4.]], device='cuda:0'), torch.Size([1, 5]), 2)

In [43]:
vT = v.T
vT, vT.shape, vT.ndim

(tensor([[0.],
         [1.],
         [2.],
         [3.],
         [4.]], device='cuda:0'),
 torch.Size([5, 1]),
 2)